In [115]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-startup-funding/startup_funding.csv


In [116]:
startup = pd.read_csv('/kaggle/input/indian-startup-funding/startup_funding.csv')
startup

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29/01/2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
3040,3041,29/01/2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30/01/2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
3042,3043,30/01/2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


# Data Cleaning

In [117]:
# Drop Remarks column
startup.drop(['Remarks'],axis=1,inplace=True)

In [118]:
# Rename the columns
startup.rename(columns = {
    'Date dd/mm/yyyy' : 'date',
    'Startup Name' : 'name',
    'Industry Vertical':'vertical',
    'SubVertical':'subvertical',
    'City  Location':'city',
    'Investors Name':'investors',
    'InvestmentnType':'type',
    'Amount in USD':'amount'
},inplace=True)

In [119]:
# Fill null values of amount column with 0's
startup['amount'] = startup['amount'].fillna('0')

In [120]:
startup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sr No        3044 non-null   int64 
 1   date         3044 non-null   object
 2   name         3044 non-null   object
 3   vertical     2873 non-null   object
 4   subvertical  2108 non-null   object
 5   city         2864 non-null   object
 6   investors    3020 non-null   object
 7   type         3040 non-null   object
 8   amount       3044 non-null   object
dtypes: int64(1), object(8)
memory usage: 214.2+ KB


In [121]:
# Change the amount column to float but before that handle some incosistency in data
startup['amount'] = startup['amount'].str.replace(',','')
startup['amount'] = startup['amount'].str.replace('undisclosed','0')
startup['amount'] = startup['amount'].str.replace('unknown','0')
startup['amount'] = startup['amount'].str.replace('Undisclosed','0')
startup = startup[startup['amount'].str.isdigit()]

In [122]:
startup['amount'] = startup['amount'].astype('float')

/tmp/ipykernel_34/3407194637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  startup['amount'] = startup['amount'].astype('float')


In [123]:
# Function to convert USD to INR (in crore)
def to_inr(usd):
    return (usd*82.84)/10000000

In [124]:
# Convert from USD to INR (in crore)
startup['amount'] = startup['amount'].apply(to_inr)

/tmp/ipykernel_34/809002476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  startup['amount'] = startup['amount'].apply(to_inr)


In [125]:
# Set Sr No as index of dataframe
startup.set_index('Sr No',inplace=True)

In [126]:
# Change the datatype of date column from object to datetime
startup['date'] = pd.to_datetime(startup['date'],errors='coerce')

/tmp/ipykernel_34/2680959192.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  startup['date'] = pd.to_datetime(startup['date'],errors='coerce')
/tmp/ipykernel_34/2680959192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  startup['date'] = pd.to_datetime(startup['date'],errors='coerce')


In [127]:
startup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3029 entries, 1 to 3044
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         3026 non-null   datetime64[ns]
 1   name         3029 non-null   object        
 2   vertical     2858 non-null   object        
 3   subvertical  2104 non-null   object        
 4   city         2849 non-null   object        
 5   investors    3005 non-null   object        
 6   type         3027 non-null   object        
 7   amount       3029 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 213.0+ KB


In [128]:
# Drop the entites which has missing value
startup = startup.dropna(subset = ['date','name','vertical','city','investors','type','amount'])

In [130]:
startup.to_csv('startup_cleaned.csv',index=False)